In [1]:
import pandas as pd
import numpy as np
import pymongo

In [2]:
# Save path to data set in a variable (.tsv files)
title_basics_file = "../Resources/title_basics.tsv"
title_akas_file = "../Resources/title_akas.tsv"
title_principals_file = "../Resources/title_principals.tsv"
title_ratings_file = "../Resources/title_ratings.tsv"
name_basics_file = "../Resources/name_basics.tsv"


In [3]:
# Read the original data files (.tsv files)
title_basics_df= pd.read_csv(title_basics_file, sep='\t')
title_basics_df

C:\Users\Joe\Anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,\N,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
...,...,...,...,...,...,...,...,...,...
6321297,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2010,\N,\N,"Action,Drama,Family"
6321298,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010,\N,\N,"Action,Drama,Family"
6321299,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010,\N,\N,"Action,Drama,Family"
6321300,tt9916856,short,The Wind,The Wind,0,2015,\N,27,Short


In [4]:
##Clean up content, drop adult movies
title_basics_clean = title_basics_df.loc[title_basics_df['isAdult']==0]

##Drop tv episodes
title_basics_clean = title_basics_clean.loc[title_basics_clean['titleType']=='movie']

##Drop unnecessary columns (isAdult)
title_basics_clean = title_basics_clean.drop(['isAdult'], axis=1)
title_basics_clean = title_basics_clean.drop(['endYear'], axis=1)
title_basics_clean = title_basics_clean.drop(['originalTitle'], axis=1)

##Rename columns
title_basics_clean = title_basics_clean.rename(columns={"tconst":"titleId","titleType":"Type"})

#Replace \N w/ NaN
title_basics_clean = title_basics_clean.replace('\\N', 'N\A')

#title_basics_clean.drop_index
#title_basics_clean = title_basics_clean.set_index(["titleId"])

title_basics_clean

,titleId,Type,primaryTitle,startYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,1894,45,Romance
145,tt0000147,movie,The Corbett-Fitzsimmons Fight,1897,20,"Documentary,News,Sport"
332,tt0000335,movie,Soldiers of the Cross,1900,N\A,"Biography,Drama"
499,tt0000502,movie,Bohemios,1905,100,N\A
571,tt0000574,movie,The Story of the Kelly Gang,1906,70,"Biography,Crime,Drama"
...,...,...,...,...,...,...
6321191,tt9916622,movie,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,N\A,Documentary
6321218,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,2007,100,Documentary
6321230,tt9916706,movie,Dankyavar Danka,2013,N\A,Comedy
6321241,tt9916730,movie,6 Gunn,2017,116,N\A


In [5]:
# Read the title_principals_file data files (.tsv files)
title_principals_df= pd.read_csv(title_principals_file, sep='\t')
title_principals_df

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Herself""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000002,1,nm0721526,director,\N,\N
4,tt0000002,2,nm1335271,composer,\N,\N
...,...,...,...,...,...,...
36468812,tt9916880,5,nm0996406,director,principal director,\N
36468813,tt9916880,6,nm1482639,writer,\N,\N
36468814,tt9916880,7,nm2586970,writer,books,\N
36468815,tt9916880,8,nm1594058,producer,producer,\N


In [6]:
##Rename columns
title_principals_df = title_principals_df.rename(columns={"tconst":"titleId","nconst":"nameId"})

##Drop unnecessary columns (ordering)
title_principals_df = title_principals_df.drop(['ordering','job'], axis=1)

#Replace \N w/ N\A
title_principals_df = title_principals_df.replace('\\N', 'N\A')

title_principals_df = title_principals_df.loc[title_principals_df['category'].str.contains("actor|actress|director")]

#title_basics_clean.drop_index
#title_principals_df = title_principals_df.set_index(["titleId"])

title_principals_df

,titleId,nameId,category,characters
1,tt0000001,nm0005690,director,N\A
3,tt0000002,nm0721526,director,N\A
5,tt0000003,nm0721526,director,N\A
9,tt0000004,nm0721526,director,N\A
11,tt0000005,nm0443482,actor,"[""Blacksmith""]"
...,...,...,...,...
36468809,tt9916880,nm0254176,actress,"[""Moody Margaret""]"
36468810,tt9916880,nm0286175,actor,"[""Dad"",""Aerobic Al"",""Nasty Nicola""]"
36468811,tt9916880,nm1052583,actress,"[""Mum"",""Tidy Ted"",""Fang""]"
36468812,tt9916880,nm0996406,director,N\A


In [7]:
# Read the original data files (.tsv files)
title_ratings_df= pd.read_csv(title_ratings_file, sep='\t')
title_ratings_df

,tconst,averageRating,numVotes
0,tt0000001,5.6,1550
1,tt0000002,6.1,186
2,tt0000003,6.5,1207
3,tt0000004,6.2,113
4,tt0000005,6.1,1934
...,...,...,...
993148,tt9916576,5.9,7
993149,tt9916578,9.1,11
993150,tt9916720,5.1,41
993151,tt9916766,6.7,11


In [8]:
##Rename columns
title_ratings_df = title_ratings_df.rename(columns={"tconst":"titleId"})

#title_basics_clean.drop_index
#title_ratings_df = title_ratings_df.set_index(["titleId"])

title_ratings_df

,titleId,averageRating,numVotes
0,tt0000001,5.6,1550
1,tt0000002,6.1,186
2,tt0000003,6.5,1207
3,tt0000004,6.2,113
4,tt0000005,6.1,1934
...,...,...,...
993148,tt9916576,5.9,7
993149,tt9916578,9.1,11
993150,tt9916720,5.1,41
993151,tt9916766,6.7,11


In [9]:
# Read the original data files (.tsv files)
name_basics_df= pd.read_csv(name_basics_file, sep='\t')
name_basics_df

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0050419,tt0053137,tt0072308,tt0043044"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0071877,tt0117057,tt0038355,tt0037382"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,producer","tt0054452,tt0049189,tt0059956,tt0057345"
3,nm0000004,John Belushi,1949,1982,"actor,writer,soundtrack","tt0077975,tt0072562,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0069467,tt0050976,tt0083922,tt0050986"
...,...,...,...,...,...,...
9706917,nm9993714,Romeo del Rosario,\N,\N,"animation_department,art_department",tt2455546
9706918,nm9993716,Essias Loberg,\N,\N,NaN,\N
9706919,nm9993717,Harikrishnan Rajan,\N,\N,cinematographer,tt8736744
9706920,nm9993718,Aayush Nair,\N,\N,cinematographer,\N


In [10]:
##Rename columns
name_basics_df = name_basics_df.rename(columns={"nconst":"nameId"})

##Drop unnecessary columns (ordering)
name_basics_df = name_basics_df.drop(['birthYear','deathYear','knownForTitles','primaryProfession'], axis=1)


#title_basics_clean.drop_index
#name_basics_df = name_basics_df.set_index(["nameId"])

name_basics_df

,nameId,primaryName
0,nm0000001,Fred Astaire
1,nm0000002,Lauren Bacall
2,nm0000003,Brigitte Bardot
3,nm0000004,John Belushi
4,nm0000005,Ingmar Bergman
...,...,...
9706917,nm9993714,Romeo del Rosario
9706918,nm9993716,Essias Loberg
9706919,nm9993717,Harikrishnan Rajan
9706920,nm9993718,Aayush Nair


In [11]:
#Replace \N w/ N\A
name_basics_df = name_basics_df.replace('\\N', 'N\A')

name_basics_df = name_basics_df.fillna('missing')

##Clean up content, drop knowForTitles='N\A'
#name_basics_df = name_basics_df.loc[name_basics_df['assocTitleId']!='N\A']
##name_basics_df = name_basics_df.loc[name_basics_df['primaryProfession']!='N\A']

name_basics_df

,nameId,primaryName
0,nm0000001,Fred Astaire
1,nm0000002,Lauren Bacall
2,nm0000003,Brigitte Bardot
3,nm0000004,John Belushi
4,nm0000005,Ingmar Bergman
...,...,...
9706917,nm9993714,Romeo del Rosario
9706918,nm9993716,Essias Loberg
9706919,nm9993717,Harikrishnan Rajan
9706920,nm9993718,Aayush Nair


In [12]:
## Merge title_basics_clean, title_ratings_df
merge_title_df = pd.merge(title_basics_clean, title_ratings_df, on="titleId", how="inner")
merge_title_df

,titleId,Type,primaryTitle,startYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000009,movie,Miss Jerry,1894,45,Romance,5.4,81
1,tt0000147,movie,The Corbett-Fitzsimmons Fight,1897,20,"Documentary,News,Sport",5.2,310
2,tt0000335,movie,Soldiers of the Cross,1900,N\A,"Biography,Drama",6.2,39
3,tt0000574,movie,The Story of the Kelly Gang,1906,70,"Biography,Crime,Drama",6.1,537
4,tt0000615,movie,Robbery Under Arms,1907,N\A,Drama,4.8,14
...,...,...,...,...,...,...,...,...
237102,tt9914644,movie,9/11: Escape from the Towers,2018,120,Documentary,8.4,27
237103,tt9914942,movie,La vida sense la Sara Amat,2019,N\A,Drama,6.9,36
237104,tt9915790,movie,Bobbyr Bondhura,2019,N\A,Family,6.9,7
237105,tt9916160,movie,Drømmeland,2019,72,Documentary,6.5,31


In [13]:
#####
# Now need to merge the dataframes together into one dataframe
# with all the required data
####

title_principals_df.head(5)


,titleId,nameId,category,characters
1,tt0000001,nm0005690,director,N\A
3,tt0000002,nm0721526,director,N\A
5,tt0000003,nm0721526,director,N\A
9,tt0000004,nm0721526,director,N\A
11,tt0000005,nm0443482,actor,"[""Blacksmith""]"


In [14]:
merge_title_df.head(5)

,titleId,Type,primaryTitle,startYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000009,movie,Miss Jerry,1894,45,Romance,5.4,81
1,tt0000147,movie,The Corbett-Fitzsimmons Fight,1897,20,"Documentary,News,Sport",5.2,310
2,tt0000335,movie,Soldiers of the Cross,1900,N\A,"Biography,Drama",6.2,39
3,tt0000574,movie,The Story of the Kelly Gang,1906,70,"Biography,Crime,Drama",6.1,537
4,tt0000615,movie,Robbery Under Arms,1907,N\A,Drama,4.8,14


In [15]:
name_basics_df.head(5)

,nameId,primaryName
0,nm0000001,Fred Astaire
1,nm0000002,Lauren Bacall
2,nm0000003,Brigitte Bardot
3,nm0000004,John Belushi
4,nm0000005,Ingmar Bergman


In [16]:
## Merge merge_title_df and title_principals_df
merge_title_df2 = pd.merge(title_principals_df, merge_title_df, on="titleId", how="inner")
merge_title_df2

,titleId,nameId,category,characters,Type,primaryTitle,startYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000009,nm0063086,actress,"[""Miss Geraldine Holbrook (Miss Jerry)""]",movie,Miss Jerry,1894,45,Romance,5.4,81
1,tt0000009,nm0183823,actor,"[""Mr. Hamilton""]",movie,Miss Jerry,1894,45,Romance,5.4,81
2,tt0000009,nm1309758,actor,"[""Chauncey Depew - the Director of the New Yor...",movie,Miss Jerry,1894,45,Romance,5.4,81
3,tt0000009,nm0085156,director,N\A,movie,Miss Jerry,1894,45,Romance,5.4,81
4,tt0000147,nm0714557,director,N\A,movie,The Corbett-Fitzsimmons Fight,1897,20,"Documentary,News,Sport",5.2,310
...,...,...,...,...,...,...,...,...,...,...,...
1145627,tt9916428,nm0422639,actress,"[""Song Qingling""]",movie,The Secret of China,2019,N\A,"Adventure,History,War",2.9,8
1145628,tt9916428,nm0910951,director,N\A,movie,The Secret of China,2019,N\A,"Adventure,History,War",2.9,8
1145629,tt9916428,nm8680851,actor,"[""Powell Editor-in-chief of US newspaper""]",movie,The Secret of China,2019,N\A,"Adventure,History,War",2.9,8
1145630,tt9916428,nm2631291,actor,"[""lee De""]",movie,The Secret of China,2019,N\A,"Adventure,History,War",2.9,8


In [17]:
## Merge merge_title_df2 and name_basics_df
movie_df = pd.merge(merge_title_df2, name_basics_df, on="nameId", how="inner")
movie_df

,titleId,nameId,category,characters,Type,primaryTitle,startYear,runtimeMinutes,genres,averageRating,numVotes,primaryName
0,tt0000009,nm0063086,actress,"[""Miss Geraldine Holbrook (Miss Jerry)""]",movie,Miss Jerry,1894,45,Romance,5.4,81,Blanche Bayliss
1,tt0000009,nm0183823,actor,"[""Mr. Hamilton""]",movie,Miss Jerry,1894,45,Romance,5.4,81,William Courtenay
2,tt0019859,nm0183823,actor,"[""Cyril Wimborne""]",movie,Evidence,1929,N\A,"Drama,Romance",6.8,13,William Courtenay
3,tt0020403,nm0183823,actor,"[""The Minister - Guillotine Sequence""]",movie,Show of Shows,1929,128,Musical,5.9,320,William Courtenay
4,tt0000009,nm1309758,actor,"[""Chauncey Depew - the Director of the New Yor...",movie,Miss Jerry,1894,45,Romance,5.4,81,Chauncey Depew
...,...,...,...,...,...,...,...,...,...,...,...,...
1145144,tt9915790,nm10538030,director,N\A,movie,Bobbyr Bondhura,2019,N\A,Family,6.9,7,Sudipa Chatterjee
1145145,tt9916160,nm5684093,director,N\A,movie,Drømmeland,2019,72,Documentary,6.5,31,Joost van der Wiel
1145146,tt9916428,nm9445072,actor,"[""Mao Ze Dong""]",movie,The Secret of China,2019,N\A,"Adventure,History,War",2.9,8,Wang Peng Kai
1145147,tt9916428,nm8594703,actor,"[""Dr. Hatem""]",movie,The Secret of China,2019,N\A,"Adventure,History,War",2.9,8,Valery Gadreau


In [39]:
movie_final_df = movie_df[['primaryName','primaryTitle','category','characters','startYear','runtimeMinutes','genres','averageRating','numVotes']]

movie_final_df = movie_final_df.rename(columns={"primaryName":"Name","primaryTitle":"movieTitle","category":"role"})

movie_final_df

,Name,movieTitle,role,characters,startYear,runtimeMinutes,genres,averageRating,numVotes
0,Blanche Bayliss,Miss Jerry,actress,"[""Miss Geraldine Holbrook (Miss Jerry)""]",1894,45,Romance,5.4,81
1,William Courtenay,Miss Jerry,actor,"[""Mr. Hamilton""]",1894,45,Romance,5.4,81
2,William Courtenay,Evidence,actor,"[""Cyril Wimborne""]",1929,N\A,"Drama,Romance",6.8,13
3,William Courtenay,Show of Shows,actor,"[""The Minister - Guillotine Sequence""]",1929,128,Musical,5.9,320
4,Chauncey Depew,Miss Jerry,actor,"[""Chauncey Depew - the Director of the New Yor...",1894,45,Romance,5.4,81
...,...,...,...,...,...,...,...,...,...
1145144,Sudipa Chatterjee,Bobbyr Bondhura,director,N\A,2019,N\A,Family,6.9,7
1145145,Joost van der Wiel,Drømmeland,director,N\A,2019,72,Documentary,6.5,31
1145146,Wang Peng Kai,The Secret of China,actor,"[""Mao Ze Dong""]",2019,N\A,"Adventure,History,War",2.9,8
1145147,Valery Gadreau,The Secret of China,actor,"[""Dr. Hatem""]",2019,N\A,"Adventure,History,War",2.9,8


In [51]:
movie_final_df = movie_final_df.loc[movie_final_df['numVotes']>89999]
movie_final_df



,Name,movieTitle,role,characters,startYear,runtimeMinutes,genres,averageRating,numVotes
1336,Michael Curtiz,Casablanca,director,N\A,1942,102,"Drama,Romance,War",8.5,487781
3052,Alfred Abel,Metropolis,actor,"[""Johann (Joh) Fredersen""]",1927,153,"Drama,Sci-Fi",8.3,148413
3328,Harry Myers,City Lights,actor,"[""An Eccentric Millionaire""]",1931,87,"Comedy,Drama,Romance",8.5,152730
7235,Charles Chaplin,The Kid,actor,"[""A Tramp""]",1921,68,"Comedy,Drama,Family",8.3,101632
7238,Charles Chaplin,The Gold Rush,actor,"[""The Lone Prospector""]",1925,95,"Adventure,Comedy,Drama",8.2,91969
...,...,...,...,...,...,...,...,...,...
1130296,Michelle La,Searching,actress,"[""Margot""]",2018,102,"Drama,Mystery,Thriller",7.7,113326
1130297,Aneesh Chaganty,Searching,director,N\A,2018,102,"Drama,Mystery,Thriller",7.7,113326
1131391,Milly Shapiro,Hereditary,actress,"[""Charlie""]",2018,127,"Drama,Horror,Mystery",7.3,181219
1131392,Ari Aster,Hereditary,director,N\A,2018,127,"Drama,Horror,Mystery",7.3,181219


In [52]:
#movie_final_df1 = movie_final_df.iloc[0:399999]
#movie_final_df2 = movie_final_df.iloc[400000:799999]
#movie_final_df3 = movie_final_df.iloc[800000:]

movie_final_df.to_json(r'movies_votes90k.json')
#movie_final_df2.to_json(r'movies2.json')
#movie_final_df3.to_json(r'movies3.json')

In [53]:
## setup mongo connection
# The default port used by MongoDB is 27017
# https://docs.mongodb.com/manual/reference/default-mongodb-port/
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define the 'moviesDB2' database in Mongo
db = client.moviesDB2

In [57]:
## panda dataframe insert to mongo for each dataframe:
##     merge_title_df
#takes about 3 min to load

movies = db.movies
movies.insert_many(movie_final_df.to_dict('records'))
#movies.insert_many(movie_final_df2.to_dict('records'))
#movies.insert_many(movie_final_df3.to_dict('records'))


In [56]:
#movies.insert_many(movie_final_df.to_dict('records'))


In [23]:
#movies.insert_many(movie_final_df3.to_dict('records'))

In [24]:
## panda dataframe insert to mongo for each dataframe:
##     name_basics_df
#takes about 5 min to load

#names = db.names
#names.insert_many(name_basics_df.to_dict('records'))

In [25]:
## panda dataframe insert to mongo for each dataframe:
##     title_principals_df
#divide into 4 sections with rows to prevent breaking our computers

In [26]:
#title_principals_df_1 = title_principals_df.iloc[0:9000000]
#principals = db.principals

#principals.insert_many(title_principals_df_1.to_dict('records'))

In [27]:
#title_principals_df_2 = title_principals_df.iloc[9000001:18000000]

#principals.insert_many(title_principals_df_2.to_dict('records'))

In [28]:
#title_principals_df_3 = title_principals_df.iloc[18000001:27000000]

#principals.insert_many(title_principals_df_3.to_dict('records'))

In [29]:
#title_principals_df_4 = title_principals_df.iloc[27000001:]

#principals.insert_many(title_principals_df_4.to_dict('records'))
